In [1]:
from mobilenetv2 import *

In [2]:
import collections
import hashlib
import os.path
import random
import re
import sys

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import keras
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input
from keras.models import Model
from keras.layers import Dense, Input, LSTM, CuDNNLSTM, Dropout, BatchNormalization, Masking, TimeDistributed,Bidirectional 
from pathlib import Path
import sklearn
from sklearn import datasets, metrics
import shutil
from keras import initializers
from keras.models import load_model, Sequential
import pickle
import random 
from keras.utils import to_categorical
from keras.preprocessing import sequence

W0627 10:56:36.145665 139830347638528 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [3]:
#model = MobileNetV2()
#model = keras.applications.mobilenet_v2.MobileNetV2()

In [4]:
#defining hyperparameters

train_dir = "human_seg_train_frames"
val_dir = "human_seg_val_frames"
test_dir = "human_seg_test_frames"
train_bottlenecks_dir = "human_seg_train_bottlenecks"
val_bottlenecks_dir = "human_seg_val_bottlenecks"
test_bottlenecks_dir = "human_seg_test_bottlenecks"

In [5]:
def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
def get_pretrained_base_model_for_bottlenecks(show_model_summary = False):
    #Arguments     : show_model_summary--> whether to show model summary/layers
    #Returns       : models-->mobilenetV2 with its last layer being 'global_average_pooling2d_1'
    
    model = MobileNetV2()
    if show_model_summary:
        model.summary()
    base_model = Model(inputs=model.input, outputs=model.get_layer('global_average_pooling2d_1').output)
    print("Base_model_loaded 100 percent")
    return base_model

In [ ]:
def compute_and_save_bottlenecks(model, image_dir, bottlenecks_dir):
    #Arguments     : model-->model to create bottlenecks
    #                image-dir--> path of directory where images are stored in class-wise folder
    #                bottlenecks_dir--> path of directory where bottlenecks will be 
    #                                   stored in text file in class-wise folder.
    #Returns       : bottlenecks_shape
    #Description   : creates and saves the bottleneck for images using mobilenetV2
    
    ensure_folder_exists(bottlenecks_dir)
    for count,filename in enumerate(Path(image_dir).glob('**/*.jpeg')):
        file_path = str(filename)
        target_path = bottlenecks_dir + file_path[file_path.find("/"):-5]
        ensure_folder_exists(os.path.split(target_path)[0])
        img = image.load_img(file_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        bottlenecks_shape = len(features[0])
        with open(target_path + '.txt', 'w') as f:
            for item in features[0]:
                f.write("%f " % item)
        if(count%2000==0):
            print("Bottlenecks Created : "+str(count))
    print("All bottlenecks saved")
    return bottlenecks_shape

In [8]:
def preprocess_bottlenecks_to_xy_numpy_array(bottlenecks_shape,x,y):
    #Description   : preprocess bottlenecks to store them in numpy array
    
    x_temp=[]
    new_x=np.ndarray((len(x),bottlenecks_shape),np.float32)
    new_y=np.ndarray((len(y)),np.int32)
    x_temp=x[0].split()
    for i,s in enumerate(x):
        x_temp=s.split()
        for j in range(len(x_temp)):
            new_x[i,j]=float((x_temp[j]))
        new_y[i]=int(y[i])
    return new_x,new_y

In [9]:
def get_xy_saved_bottlenecks(bottlenecks_dir,bottlenecks_shape=1280):
    #Arguments     : bottlenecks_dir--> path of directory where bottlenecks will be 
    #                                   stored in text file in class-wise folder.
    #Returns       : 
    #Description   : X-->bottlenecks of all frames
    #                Y-->class of all frames in number form
    #                class_names-->List containing class names
    #                num_classes-->Number of total classes
    #                file_names-->list of names of bottlenecks file
    
    features=sklearn.datasets.load_files(bottlenecks_dir) #check its's documentation to understand what it's returning
    x=features.data
    y=features.target
    class_names=np.asarray(features.target_names)
    num_classes=len(class_names)
    X,Y=preprocess_bottlenecks_to_xy_numpy_array(bottlenecks_shape,x,y)
    file_names_with_dir=np.asarray(features.filenames)
    file_names=[]
    for s in file_names_with_dir:
        name=s.split('/')[2]
        file_names.append(name)
    return X,Y,class_names,num_classes,file_names

In [10]:
def make_pickle_file(bottlenecks_dir,output_file):
    #Arguments     : bottlenecks_dir--> path of directory where bottlenecks will be 
    #                                   stored in text file in class-wise folder.
    #                output_file--> path where pickle file need to be stored
    #Returns       : None
    #Description   : Loading bottlenecks from txt file again and again takes too much time.
    #                That's why we store all bottleneck vallues in pickle file that can be loaded easily
    
    X,Y,class_names,num_classes,file_names=get_xy_saved_bottlenecks(bottlenecks_dir)
    cache={
        'features':X,
        'class_value':Y,
        'class_names':class_names,
        'file_names':file_names,
        'num_classes':num_classes
    }
    out_file=output_file
    with open(out_file, 'wb') as fout:
        pickle.dump(cache, fout)

In [ ]:
base_model=get_pretrained_base_model_for_bottlenecks(show_model_summary = False)
bottlenecks_shape=compute_and_save_bottlenecks(base_model, train_dir, train_bottlenecks_dir)
bottlenecks_shape=compute_and_save_bottlenecks(base_model, val_dir, val_bottlenecks_dir)
bottlenecks_shape=compute_and_save_bottlenecks(base_model, test_dir, test_bottlenecks_dir)
make_pickle_file(train_bottlenecks_dir,output_file="human_seg_train_features.pkl")
make_pickle_file(val_bottlenecks_dir,output_file="human_seg_val_features.pkl")
make_pickle_file(test_bottlenecks_dir,output_file="human_seg_test_features.pkl")
print("bottlenecks_shape="+str(bottlenecks_shape))
remove_ipynb_checkpoint_files(bottlenecks_dir)

In [12]:
# Now we'll create and train RNN model

In [12]:
def get_data_from_pickle_file(file_name):
    #Arguments     : pickle file path
    #Returns       : features, Y, class_names, file_names, num_classes
    #Description   : Loads data from pickle file
    
    pickle_in = open(file_name,"rb")
    cache = pickle.load(pickle_in)
    features=cache['features']
    Y=cache['class_value']
    class_names=cache['class_names']
    file_names=np.asarray(cache['file_names'])
    num_classes=cache['num_classes']
    print("Data loaded from pickle file")
    return features, Y, class_names, file_names, num_classes

In [13]:
def preprocess_data_for_rnn_input(features,Y,file_names):
    #Arguments     : features--> x for rnn input
    #                Y,file_names
    #Description   : as all frames are in random order but we need to give rnn model frames
    #                in video wise order. So, this function clubs all frames of particular video 
    #                together and then sorts them according to their filenames so that they can be
    #                given as input to rnn in series.
    
    zipped=zip(file_names,Y,features)
    zipped=sorted(zipped,key = lambda x: x[0])
    total_frames=Y.shape[0]
    feature_size=features.shape[1]
    rnn_X=[]
    rnn_Y=[]    
    rnn_file_names=[]

    start=0
    end=0
    current_name=""
    last_name=""
    while end<total_frames:
        if last_name=="":
            temp_name=zipped[0][0]
            last_name=temp_name[:-len(temp_name.split("_")[-1])] 
            current_name=temp_name[:-len(temp_name.split("_")[-1])] 
        while current_name==last_name:
            end+=1
            if end>=total_frames:
                break
            current_name=zipped[end][0][:-len(zipped[end][0].split("_")[-1])] 
        current_zip=zipped[start:end]
        current_zip=sorted(current_zip,key = lambda x : int((x[0].split('_')[-1]).split('.')[0]))
        rnn_X.append(list(zip(*current_zip))[2])
        rnn_Y.append((list(zip(*current_zip))[1])[0])
        rnn_file_names.append(list(zip(*current_zip))[0])
        last_name=current_name
        start=end
    rnn_X=np.asarray(rnn_X)
    rnn_Y=np.asarray(rnn_Y)
    rnn_file_names=np.asarray(rnn_file_names)
    return rnn_X,rnn_Y,rnn_file_names

In [15]:
def shuffle_data(x,y,file_name):
    #Description   : shuffles the data video-wise ( not frame-wise)
    
    np.random.seed(25)  
    random=np.arange(x.shape[0])
    np.random.shuffle(random)
    x=x[random]
    y=y[random]
    file_name=file_name[random]
    del random
    return x,y,file_name

In [16]:
#Getting data from pickle file
features,Y,class_names,file_names,num_classes=get_data_from_pickle_file(
                                              file_name="human_seg_train_features.pkl")
val_features,val_Y,val_class_names,val_file_names,val_num_classes=get_data_from_pickle_file(
                                                                  file_name="human_seg_val_features.pkl")
test_features,test_Y,test_class_names,test_file_names,test_num_classes=get_data_from_pickle_file(
                                                                  file_name="human_seg_test_features.pkl")
#preprocess
rnn_X,rnn_Y,rnn_file_names=preprocess_data_for_rnn_input(features,Y,file_names)
val_rnn_X,val_rnn_Y,val_rnn_file_names=preprocess_data_for_rnn_input(val_features,val_Y,val_file_names)
test_rnn_X,test_rnn_Y,test_rnn_file_names=preprocess_data_for_rnn_input(test_features,test_Y,test_file_names)
#shuffle
rnn_X,rnn_Y,file_names=shuffle_data(rnn_X,rnn_Y,file_names)
#val_rnn_X,val_rnn_Y,val_file_names=shuffle_data(val_rnn_X,val_rnn_Y,val_file_names)
#test_rnn_X,test_rnn_Y,test_file_names=shuffle_data(test_rnn_X,test_rnn_Y,test_file_names)
#To categorical classes
val_temp_y=val_rnn_Y
test_temp_y=test_rnn_Y
rnn_Y=to_categorical(rnn_Y)
val_rnn_Y=to_categorical(val_rnn_Y)
test_rnn_Y=to_categorical(test_rnn_Y)

Data loaded from pickle file
Data loaded from pickle file
Data loaded from pickle file


In [18]:
#for epoch=18, approx accuracy stable=45
def lstm_model_1(num_classes):
    model = Sequential()
    model.add(Masking(mask_value=0.0,input_shape=(None,1280)))
    model.add(TimeDistributed(Dense(128)))
    model.add(LSTM(256,dropout=0.5)) 
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    return model

In [19]:
#for epoch=18, approx accuracy stable>=50
def lstm_model_2(num_classes):
    model = Sequential()
    model.add(Masking(mask_value=0.0,input_shape=(None,1280)))
    model.add(Bidirectional(LSTM(512,dropout=0.5)))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    return model

In [20]:
#for epoch=18, approx accuracy stable>=50
def lstm_model(num_classes):
    model = Sequential()
    model.add(Masking(mask_value=0.0,input_shape=(None,1280)))
    model.add(Bidirectional(LSTM(512,dropout=0.5,return_sequences=True)))
    model.add(Bidirectional(LSTM(256,dropout=0.5)))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    return model

In [21]:
def evaluate_for_validation(x,y,model,keyword):
    correct=0
    total_loss=0
    y_pred=[]
    for i in range(x.shape[0]):
        batch_X = sequence.pad_sequences(x[i], dtype='float32', padding='pre', value=0.0)
        batch_X=np.reshape(batch_X,(1,batch_X.shape[0],batch_X.shape[1]))
        batch_Y=np.reshape(y[i,:],(1,y.shape[1]))
        y_pred.append(model.predict_on_batch(batch_X))
        loss,acc=model.evaluate(batch_X, batch_Y,verbose=0)
        total_loss+=loss
        if int(acc)==1:
            correct+=1
    total_loss/=x.shape[0]
    percent=correct*100/x.shape[0]
    if keyword=="val":
        print("Validation Accuracy = ",str(percent)," %","     Validation Loss = ",str(total_loss))
    else:
        print("      Test Accuracy = ",str(percent)," %","           Test Loss = ",str(total_loss))
    return y_pred

In [22]:
model = lstm_model(num_classes=16)
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


W0627 10:56:52.910196 139830347638528 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0627 10:56:54.659363 139830347638528 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [32]:
epochs=1
batch_size=128
total_videos=rnn_X.shape[0]
for i in range(epochs):
    start=0
    end=batch_size
    count=0
    loss=0
    accuracy=0
    while(end<=total_videos):
        batch_X=rnn_X[start:end]
        batch_Y=rnn_Y[start:end,:]
        batch_X = sequence.pad_sequences(batch_X, dtype='float32', padding='pre', value=0.0)
        cur_loss,cur_accuracy=model.train_on_batch(batch_X,batch_Y)
        loss+=cur_loss
        accuracy+=cur_accuracy
        start=end
        end+=batch_size
        count+=1
    batch_X=rnn_X[start:]
    batch_Y=rnn_Y[start:,:]
    batch_X = sequence.pad_sequences(batch_X, dtype='float32', padding='pre', value=0.0)
    
    cur_loss,cur_accuracy=model.train_on_batch(batch_X,batch_Y)
    #loss+=cur_loss
    #accuracy+=cur_accuracy
    #count+=1
    loss/=count
    accuracy/=count
    print("Epoch-->",str(i))
    print("Train Loss = ",str(loss),"      Train Accuracy = ",str(accuracy))
    evaluate_for_validation(val_rnn_X,val_rnn_Y,model,keyword="val")
    evaluate_for_validation(test_rnn_X,test_rnn_Y,model,keyword="test")

Epoch--> 0
Train Loss =  0.1653456191221873       Train Accuracy =  0.9430338541666666
Validation Accuracy =  78.125  %      Validation Loss =  1.017340524912015
      Test Accuracy =  65.625  %            Test Loss =  1.2467592750572933


In [36]:
model.save("human_seg_rnn_mask_model.h5")

In [23]:
model = load_model("human_seg_rnn_mask_model.h5")

Instructions for updating:
Use tf.cast instead.


W0627 10:57:11.794563 139830347638528 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [33]:
y_pred=evaluate_for_validation(val_rnn_X,val_rnn_Y,model,keyword="val")

Validation Accuracy =  78.125  %      Validation Loss =  1.017340524912015


In [34]:
y_pred=np.asarray(y_pred)
y_pred=np.reshape(y_pred,(32,16))
print(y_pred.max(axis=-1))
print(class_names)
y_classes = y_pred.argmax(axis=-1)
zipped1=list(zip(val_temp_y,y_classes))
for i in range(len(zipped1)):
    print(class_names[int(zipped1[i][0])],"------------>",class_names[int(zipped1[i][1])])
matrix = metrics.confusion_matrix(val_rnn_Y.argmax(axis=-1), y_pred.argmax(axis=-1))
matrix

[0.99976593 0.46000385 0.864835   0.93600845 0.9996493  0.9331826
 0.5098977  0.5469188  0.97864836 0.9914725  0.97451425 0.87044996
 0.77866656 0.5944352  0.67526996 0.9979905  0.99888057 0.9989371
 0.9972905  0.9316171  0.99775547 0.99714595 0.8912208  0.86364
 0.65579826 0.84527004 0.9976956  0.99947363 0.9939129  0.83029974
 0.88435155 0.9874193 ]
['ABOVE' 'ACCIDENT' 'ADVISE_or_INFLUENCE' 'AFTERNOON' 'AGAIN' 'ALLIGATOR'
 'ANSWER' 'A_LOT' 'BECAME' 'COME_HERE' 'EXCEED' 'GENERATION' 'LESS_THAN'
 'OF' 'THREAT' 'TOTAL']
ABOVE ------------> ABOVE
ABOVE ------------> ABOVE
TOTAL ------------> ACCIDENT
TOTAL ------------> TOTAL
OF ------------> OF
OF ------------> OF
ANSWER ------------> ANSWER
ANSWER ------------> ANSWER
COME_HERE ------------> COME_HERE
COME_HERE ------------> COME_HERE
THREAT ------------> THREAT
THREAT ------------> THREAT
ALLIGATOR ------------> ALLIGATOR
ALLIGATOR ------------> ALLIGATOR
GENERATION ------------> ADVISE_or_INFLUENCE
GENERATION ------------> GENERATION

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [35]:
y_pred=evaluate_for_validation(test_rnn_X,test_rnn_Y,model,keyword="test")
y_pred=np.asarray(y_pred)
y_pred=np.reshape(y_pred,(32,16))
print(y_pred.max(axis=-1))
print(class_names)
y_classes = y_pred.argmax(axis=-1)
zipped1=list(zip(test_temp_y,y_classes))
for i in range(len(zipped1)):
    print(class_names[int(zipped1[i][0])],"------------>",class_names[int(zipped1[i][1])])
matrix = metrics.confusion_matrix(val_rnn_Y.argmax(axis=-1), y_pred.argmax(axis=-1))
matrix

      Test Accuracy =  65.625  %            Test Loss =  1.2467592750572933
[0.51499397 0.8116202  0.6549879  0.9762674  0.6230682  0.69542474
 0.99947053 0.70946366 0.99055547 0.99984336 0.997203   0.90080386
 0.9992248  0.9777108  0.9847141  0.999879   0.9866209  0.99681205
 0.9611812  0.99702317 0.72886205 0.72410405 0.53693587 0.87503856
 0.99342597 0.998727   0.999998   0.9999175  0.9876281  0.9997695
 0.9614989  0.8279593 ]
['ABOVE' 'ACCIDENT' 'ADVISE_or_INFLUENCE' 'AFTERNOON' 'AGAIN' 'ALLIGATOR'
 'ANSWER' 'A_LOT' 'BECAME' 'COME_HERE' 'EXCEED' 'GENERATION' 'LESS_THAN'
 'OF' 'THREAT' 'TOTAL']
ABOVE ------------> ABOVE
ABOVE ------------> ABOVE
TOTAL ------------> TOTAL
TOTAL ------------> ACCIDENT
OF ------------> COME_HERE
OF ------------> ABOVE
ANSWER ------------> ANSWER
ANSWER ------------> GENERATION
COME_HERE ------------> ACCIDENT
COME_HERE ------------> COME_HERE
THREAT ------------> THREAT
THREAT ------------> GENERATION
ALLIGATOR ------------> AFTERNOON
ALLIGATOR -------

array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])